In [1]:
import argparse

import matplotlib.pyplot as plt
import cartopy.crs as ccrs

import cartopy.feature as cfeature
import shapely.geometry as sgeom

parser = argparse.ArgumentParser()
parser.add_argument("--grid-path",default="/lustre_scratch/shaerdan/maps/EOCIS-CHUK-GRID-100M-v1.0.nc")
parser.add_argument("--input-folder",default="/lustre_scratch/shaerdan/data_folder/train")
args = parser.parse_args()

path = args.grid_path

import os
import xarray as xr

# open the target grid, just to get the UK bounding box
ds = xr.open_dataset(path)

# work out the area of interest - the UK
x_min = 0
x_max = int(ds.x.max())
y_min = 0
y_max = int(ds.y.max())

print(x_min,x_max,y_min,y_max)

# collect the input files, from each file we will read multiple scenes and bin them
input_files = []
for file in os.listdir(args.input_folder):
    if file.endswith(".nc"):
        input_files.append(os.path.join(args.input_folder,file))

fig = plt.figure(figsize=(12,24))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.OSGB())
ax.set_extent([x_min, x_max, y_min, y_max], crs=ccrs.OSGB())

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)

colour = "#00FF0010"

# record the number of scenes that fall into each (y,x) box
counts = {}

# track the maximum number of scenes in any bin
max_count = 0

# define the bin size in m
sz=10000

for file in input_files:
    # for each input file...
    print(file)
    ds = xr.open_dataset(file)
    n = ds.x_eastings.shape[0]
    # go through the scenes in the file...
    for i in range(0,n):
        eastings = ds.x_eastings
        northings = ds.y_northings

        # for each scene bin the x,y coordinates
        grid_x_min = round(float(eastings[i,0]+5000)/sz)*sz
        grid_y_min = round(float(northings[i,-1]+5000)/sz)*sz
        
        # update the count for this bin
        if (grid_y_min,grid_x_min) not in counts:
            counts[(grid_y_min,grid_x_min)] = 0
        count = counts[(grid_y_min, grid_x_min)] + 1
        if count > max_count:
            max_count = count
        counts[(grid_y_min,grid_x_min)] = count

# for each bin, plot a rectangle on the map
for (y,x) in counts:
    count = counts[(y,x)]
    # define a red colour with transparency (alpha channel) based on the count
    col = "#FF0000%02X"%(int(255*(count/float(max_count))))
    polygon = sgeom.Polygon(shell=[(x-sz/2,y-sz/2),(x-sz/2,y+sz/2),(x+sz/2,y+sz/2),(x+sz/2,y-sz/2)])
    ax.add_geometries([polygon], ccrs.OSGB(),facecolor=col)

plt.savefig("plot.png")

usage: ipykernel_launcher.py [-h] [--grid-path GRID_PATH]
                             [--input-folder INPUT_FOLDER]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/jovyan/.local/share/jupyter/runtime/kernel-f0c87ad4-d33b-4bac-84e6-9bfd0a300c7e.json


SystemExit: 2

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
